### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [5]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
#filter_query = 'section_name:%Movies" AND type_of_material:"Review" AND headline:"love"'

#to the reviewers: maybe I missed this, but the line above (the one given in the started code) does not work in the NYT API.
#when you build a NYT API call, you will see the followiong characters used in their filter_query string
#the part I may have missed is how the double quotes and spaces above get converted into the "%3A/%20/%22"
#characrters I see in the filter_query below (which I have confiermed works) - maybe therte is a "conversion"
#function in oythin that I wasn;t aware of?
filter_query = 'section_name%3A%22Movies%22%20AND%20type_of_material%3A%22Review%22%20AND%20headline%3A%22love%22'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (f'{url}api-key={nyt_api_key}&begin_date={begin_date}&end_Date={end_date}'
             +f'&fq={filter_query}&sort={sort}&fl={field_list}')

In [6]:
# Create an empty list to store the reviews
review_list = []

# loop through pages 0-19
for i in range(0,20):
    # create query with a page number
    url = query_url+'&page='+(str(i))
    #print(url)

    # API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(url).json()

    # Print the page that was just retrieved
    print(f'Checking Page {i}')
    #print(json.dumps(results, indent = 4))

    # Print the page number that had no results then break from the loop
    if len(reviews['response']['docs']) == 0:
        print(f'Page {i} has no entries')
        break
    else:
        # Try and save the reviews to the reviews_list
        try:
            # loop through the reviews["response"]["docs"] and append each review to the list
            for review in reviews['response']['docs']:
                    review_list.append(review)
        except:
            print(f'Error in review extraction loop on page {i}')
            continue
    
    # Add a twelve second interval between queries to stay within API query limits
    # to the reviewers: the time delay should come AFTER a call has been made AND processed
    # where this delay was placed in the starter code was AFTER the call, but BEFOREE the
    # results of the call were processed (which is done locally).
    # placing this delay here will do what is intended - after a call (and processing the results)
    # there will be a delay before the next call
    time.sleep(12)

Checking Page 0
Checking Page 1
Checking Page 2
Checking Page 3
Checking Page 4
Checking Page 5
Checking Page 6
Checking Page 7
Checking Page 8
Checking Page 9
Checking Page 10
Checking Page 11
Checking Page 12
Checking Page 13
Checking Page 14
Checking Page 15
Checking Page 16
Checking Page 17
Checking Page 18
Checking Page 19


In [7]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(review_list[0:5], indent = 4))

[
    {
        "web_url": "https://www.nytimes.com/2024/07/12/movies/martin-scorsese-made-in-england.html",
        "snippet": "\u201cMade in England\u201d is an essay film about the artists whose passion and cinematography deeply influenced the American director.",
        "source": "The New York Times",
        "headline": {
            "main": "From Scorsese With Love: A Tribute to Powell-Pressburger Movies",
            "kicker": "Documentary Lens",
            "content_kicker": null,
            "print_headline": "Scorsese\u2019s Love Letter to Two English Filmmakers",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Documentary Films and Programs",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "subject",
                "value": "Movies",
                "rank": 2,
                "maj

In [8]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(review_list)
reviews_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2024/07/12/movies/mart...,“Made in England” is an essay film about the a...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2024-07-12T09:03:06+0000,479,From Scorsese With Love: A Tribute to Powell-P...,Documentary Lens,None,Scorsese’s Love Letter to Two English Filmmakers,None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None
1,https://www.nytimes.com/2024/07/11/movies/fly-...,"Greg Berlanti’s movie, starring Scarlett Johan...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-07-11T11:00:22+0000,525,‘Fly Me to the Moon’ Review: This NASA Rom-Com...,None,None,"It’s Love, NASA Style, But Earthbound in the End",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
2,https://www.nytimes.com/2024/05/30/movies/hand...,A zombie movie is wrapped in a gentle tale of ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-05-30T09:01:06+0000,824,‘Handling the Undead’ Review: When the Dead Do...,Critic’s Pick,None,Dead Loved Ones Have Returned. Don’t Rejoice Yet.,None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None
3,https://www.nytimes.com/2024/04/25/movies/chal...,"Zendaya, Josh O’Connor and Mike Faist play fri...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-04-25T17:15:47+0000,1029,"‘Challengers’ Review: Game, Set, Love Matches",Critic’s pick,None,"Triple Fault: Game, Set, Love Matches",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
4,https://www.nytimes.com/2024/04/25/movies/unsu...,"In fact, there’s a lot of singing in the clan ...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-04-25T09:04:35+0000,276,‘Unsung Hero’ Review: Music Dedicated to the O...,None,None,Unsung Hero,None,None,None,By Nicolas Rapold,"[{'firstname': 'Nicolas', 'middlename': None, ...",None


In [9]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

# reviews_df['title'] = reviews_df['headline.main'].map(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

# Note to reviewer: I noticed that the movie titles are not always in the format of "'Movie title' Review:..."
# so a more robust logic needs to be implemented in order to find movie titles either buried in the headline.main
# column or some other column within the dataframe (such as snippet)
# here is another example of a different title placement: "Review: In ‘Keep the Change,’ Two People With ..."
# for now I have implemented the minimum solutiuon for this step, as described

err_title = 'No movie title found for this row'

def extract_movie_title(movie_string):
    beg_pos = movie_string.find('\u2018')
    end_pos = movie_string.find('\u2019 Review')
    
    if end_pos == -1:
        return (err_title)
    else:
        return (movie_string[beg_pos + 1:end_pos])

reviews_df['title'] = reviews_df['headline.main'].map(lambda x: extract_movie_title(x))

reviews_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2024/07/12/movies/mart...,“Made in England” is an essay film about the a...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2024-07-12T09:03:06+0000,479,From Scorsese With Love: A Tribute to Powell-P...,Documentary Lens,None,Scorsese’s Love Letter to Two English Filmmakers,None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None,No movie title found for this row
1,https://www.nytimes.com/2024/07/11/movies/fly-...,"Greg Berlanti’s movie, starring Scarlett Johan...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-07-11T11:00:22+0000,525,‘Fly Me to the Moon’ Review: This NASA Rom-Com...,None,None,"It’s Love, NASA Style, But Earthbound in the End",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,Fly Me to the Moon
2,https://www.nytimes.com/2024/05/30/movies/hand...,A zombie movie is wrapped in a gentle tale of ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-05-30T09:01:06+0000,824,‘Handling the Undead’ Review: When the Dead Do...,Critic’s Pick,None,Dead Loved Ones Have Returned. Don’t Rejoice Yet.,None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None,Handling the Undead
3,https://www.nytimes.com/2024/04/25/movies/chal...,"Zendaya, Josh O’Connor and Mike Faist play fri...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-04-25T17:15:47+0000,1029,"‘Challengers’ Review: Game, Set, Love Matches",Critic’s pick,None,"Triple Fault: Game, Set, Love Matches",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Challengers
4,https://www.nytimes.com/2024/04/25/movies/unsu...,"In fact, there’s a lot of singing in the clan ...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-04-25T09:04:35+0000,276,‘Unsung Hero’ Review: Music Dedicated to the O...,None,None,Unsung Hero,None,None,None,By Nicolas Rapold,"[{'firstname': 'Nicolas', 'middlename': None, ...",None,Unsung Hero


In [10]:
# to the reviewers: I've put this line in to check to see how many titles were not extracted
# from the 'main' column, as commented above
reviews_df['title'].value_counts()

title
No movie title found for this row       29
Dick Johnson Is Dead                     1
Sylvie’s Love                            1
Ariana Grande: Excuse Me, I Love You     1
Museum Town                              1
                                        ..
Both Sides of the Blade                  1
Fire of Love                             1
Love & Gelato                            1
Stay Prayed Up                           1
In a Relationship                        1
Name: count, Length: 172, dtype: int64

In [11]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].map(lambda x: extract_keywords(x))

reviews_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2024/07/12/movies/mart...,“Made in England” is an essay film about the a...,The New York Times,subject: Documentary Films and Programs;subjec...,2024-07-12T09:03:06+0000,479,From Scorsese With Love: A Tribute to Powell-P...,Documentary Lens,None,Scorsese’s Love Letter to Two English Filmmakers,None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None,No movie title found for this row
1,https://www.nytimes.com/2024/07/11/movies/fly-...,"Greg Berlanti’s movie, starring Scarlett Johan...",The New York Times,"subject: Movies;persons: Harrelson, Woody;pers...",2024-07-11T11:00:22+0000,525,‘Fly Me to the Moon’ Review: This NASA Rom-Com...,None,None,"It’s Love, NASA Style, But Earthbound in the End",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,Fly Me to the Moon
2,https://www.nytimes.com/2024/05/30/movies/hand...,A zombie movie is wrapped in a gentle tale of ...,The New York Times,"subject: Movies;persons: Reinsve, Renate;perso...",2024-05-30T09:01:06+0000,824,‘Handling the Undead’ Review: When the Dead Do...,Critic’s Pick,None,Dead Loved Ones Have Returned. Don’t Rejoice Yet.,None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None,Handling the Undead
3,https://www.nytimes.com/2024/04/25/movies/chal...,"Zendaya, Josh O’Connor and Mike Faist play fri...",The New York Times,subject: Movies;creative_works: Challengers (M...,2024-04-25T17:15:47+0000,1029,"‘Challengers’ Review: Game, Set, Love Matches",Critic’s pick,None,"Triple Fault: Game, Set, Love Matches",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Challengers
4,https://www.nytimes.com/2024/04/25/movies/unsu...,"In fact, there’s a lot of singing in the clan ...",The New York Times,subject: Movies;creative_works: Unsung Hero (M...,2024-04-25T09:04:35+0000,276,‘Unsung Hero’ Review: Music Dedicated to the O...,None,None,Unsung Hero,None,None,None,By Nicolas Rapold,"[{'firstname': 'Nicolas', 'middlename': None, ...",None,Unsung Hero


In [12]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
movie_title_list = reviews_df['title'].to_list()

### Access The Movie Database API

In [14]:
# Prepare The Movie Database query
title_query_url = "https://api.themoviedb.org/3/search/movie?query="
movie_query_url = "https://api.themoviedb.org/3/movie/"
tmdb_key_string = "&api_key=" + tmdb_api_key
tmdb_key_movie_search = "?api_key=" + tmdb_api_key

In [15]:
def extract_values(keyword_list, col_to_extract):
    extracted_names = []
    for item in keyword_list:
        # Extract 'name'
        keyword = f"{item[col_to_extract]}" 
        # Append the keyword item to the extracted_keywords list
        extracted_names.append(keyword)
    return extracted_names

# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in movie_title_list:
    
    # Check if we need to sleep before making a request
    if (request_counter % 50) == 0:
        print('Application sleeping')
        time.sleep(12)
        print('Application resuming')
    
    # Perform a "GET" request for The Movie Database
    request_string = (f'{title_query_url}{title}{tmdb_key_string}')
    results_json = requests.get(request_string).json()

    # Add 1 to the request counter
    request_counter += 1

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = results_json['results'][0]['id']

        # Make a request for a the full movie details
        movie_request_string = (f'{movie_query_url}{movie_id}{tmdb_key_movie_search}')

        # Execute "GET" request with url
        movie_details = requests.get(movie_request_string).json()

        # Print out the title that was found
        print(f'Found: {title}')
        
        # Extract the genre_names into a list
        genres = extract_values(movie_details['genres'], 'name')

        # Extract the spoken_languages' English name into a list
        spoken_languages = extract_values(movie_details['spoken_languages'], 'english_name')

        # Extract the production_countries' name into a list
        production_countries = extract_values(movie_details['production_countries'], 'name')
        
        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        
        dict = ({
            'title':title,
            'genres':genres,
            'spoken_languages':spoken_languages,
            'production_countries':production_countries, 
            'original_title':movie_details['original_title'],
            'budget':movie_details['budget'],
            'original_language':movie_details['original_language'],
            'homepage':movie_details['homepage'],
            'overview':movie_details['overview'],
            'popularity':movie_details['popularity'],
            'runtime':movie_details['runtime'],
            'revenue':movie_details['revenue'],
            'release_date':movie_details['release_date'],
            'vote_average':movie_details['vote_average'],
            'vote_count':movie_details['vote_count']
        })
        
        tmdb_movies_list.append(dict)
        
        print(f'{title} Added to tmdb_movies_list')
        
    except:
        print(f'Not found: {title}')


Not found: No movie title found for this row
Found: Fly Me to the Moon
Fly Me to the Moon Added to tmdb_movies_list
Found: Handling the Undead
Handling the Undead Added to tmdb_movies_list
Found: Challengers
Challengers Added to tmdb_movies_list
Found: Unsung Hero
Unsung Hero Added to tmdb_movies_list
Found: Love Lies Bleeding
Love Lies Bleeding Added to tmdb_movies_list
Found: Bleeding Love
Bleeding Love Added to tmdb_movies_list
Found: Bob Marley: One Love
Bob Marley: One Love Added to tmdb_movies_list
Found: Molli and Max in the Future
Molli and Max in the Future Added to tmdb_movies_list
Found: The Taste of Things
The Taste of Things Added to tmdb_movies_list
Found: Pictures of Ghosts
Pictures of Ghosts Added to tmdb_movies_list
Found: Anyone but You
Anyone but You Added to tmdb_movies_list
Found: Fallen Leaves
Fallen Leaves Added to tmdb_movies_list
Found: Radical
Radical Added to tmdb_movies_list
Found: Fingernails
Fingernails Added to tmdb_movies_list
Found: My Love Affair With 

In [16]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[0:5], indent = 4))

[
    {
        "title": "Fly Me to the Moon",
        "genres": [
            "Comedy",
            "Romance"
        ],
        "spoken_languages": [
            "English"
        ],
        "production_countries": [
            "United States of America"
        ],
        "original_title": "Fly Me to the Moon",
        "budget": 100000000,
        "original_language": "en",
        "homepage": "https://www.flymetothemoon.movie",
        "overview": "Sparks fly in all directions as marketing maven Kelly Jones, brought in to fix NASA's public image, wreaks havoc on Apollo 11 launch director Cole Davis' already difficult task of putting a man on the moon. When the White House deems the mission too important to fail, Jones is directed to stage a fake moon landing as backup, and the countdown truly begins.",
        "popularity": 89.17,
        "runtime": 132,
        "revenue": 37296640,
        "release_date": "2024-07-06",
        "vote_average": 7.209,
        "vote_count": 127
    

In [17]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df.head(5)

,title,genres,spoken_languages,production_countries,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count
0,Fly Me to the Moon,"[Comedy, Romance]",[English],[United States of America],Fly Me to the Moon,100000000,en,https://www.flymetothemoon.movie,Sparks fly in all directions as marketing mave...,89.170,132,37296640,2024-07-06,7.209,127
1,Handling the Undead,"[Horror, Drama, Mystery]","[Norwegian, Swedish]","[Greece, Norway, Sweden]",Håndtering av udøde,0,no,https://trustnordisk.com/movie/handling-the-un...,"On a hot summer day in Oslo, the dead mysterio...",280.446,98,105046,2024-02-09,6.620,46
2,Challengers,"[Romance, Drama]","[English, Romanian]","[Italy, United States of America]",Challengers,55000000,en,https://www.amazon.com/salp/challengers,Tennis player turned coach Tashi has taken her...,128.738,132,94182533,2024-04-18,7.175,1196
3,Unsung Hero,"[Drama, Music, Family]",[English],[United States of America],Unsung Hero,6000000,en,https://unsunghero.movie/,When David Smallbone's successful music compan...,87.539,113,20190557,2024-04-26,6.521,24
4,Love Lies Bleeding,"[Crime, Romance, Thriller]",[English],"[United Kingdom, United States of America]",Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,189.736,104,12027893,2024-03-08,6.583,400


### Merge and Clean the Data for Export

In [19]:
# Merge the New York Times reviews and TMDB DataFrames on title
combined_df = tmdb_df.merge(reviews_df, on=['title'])
combined_df.head(5)

,title,genres,spoken_languages,production_countries,original_title,budget,original_language,homepage,overview,popularity,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,Fly Me to the Moon,"[Comedy, Romance]",[English],[United States of America],Fly Me to the Moon,100000000,en,https://www.flymetothemoon.movie,Sparks fly in all directions as marketing mave...,89.170,...,‘Fly Me to the Moon’ Review: This NASA Rom-Com...,None,None,"It’s Love, NASA Style, But Earthbound in the End",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
1,Handling the Undead,"[Horror, Drama, Mystery]","[Norwegian, Swedish]","[Greece, Norway, Sweden]",Håndtering av udøde,0,no,https://trustnordisk.com/movie/handling-the-un...,"On a hot summer day in Oslo, the dead mysterio...",280.446,...,‘Handling the Undead’ Review: When the Dead Do...,Critic’s Pick,None,Dead Loved Ones Have Returned. Don’t Rejoice Yet.,None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None
2,Challengers,"[Romance, Drama]","[English, Romanian]","[Italy, United States of America]",Challengers,55000000,en,https://www.amazon.com/salp/challengers,Tennis player turned coach Tashi has taken her...,128.738,...,"‘Challengers’ Review: Game, Set, Love Matches",Critic’s pick,None,"Triple Fault: Game, Set, Love Matches",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
3,Unsung Hero,"[Drama, Music, Family]",[English],[United States of America],Unsung Hero,6000000,en,https://unsunghero.movie/,When David Smallbone's successful music compan...,87.539,...,‘Unsung Hero’ Review: Music Dedicated to the O...,None,None,Unsung Hero,None,None,None,By Nicolas Rapold,"[{'firstname': 'Nicolas', 'middlename': None, ...",None
4,Love Lies Bleeding,"[Crime, Romance, Thriller]",[English],"[United Kingdom, United States of America]",Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,189.736,...,"‘Love Lies Bleeding’ Review: Kristen Stewart, ...",Critic’s Pick,None,"Bad Romance, Hard-Boiled Style",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [20]:
def remove_chars(original_str, char_list):
    for ch in char_list:
        original_str = original_str.replace(ch, '')
    return original_str

# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for col in columns_to_fix:
    # Convert the column to type 'str'
    combined_df[col] = combined_df[col].astype('str')
    # Loop through characters to remove
    combined_df[col] = combined_df[col].map(lambda x: remove_chars(x, characters_to_remove))

# Display the fixed DataFrame
combined_df.head(5)

,title,genres,spoken_languages,production_countries,original_title,budget,original_language,homepage,overview,popularity,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,Fly Me to the Moon,"Comedy, Romance",English,United States of America,Fly Me to the Moon,100000000,en,https://www.flymetothemoon.movie,Sparks fly in all directions as marketing mave...,89.170,...,‘Fly Me to the Moon’ Review: This NASA Rom-Com...,None,None,"It’s Love, NASA Style, But Earthbound in the End",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
1,Handling the Undead,"Horror, Drama, Mystery","Norwegian, Swedish","Greece, Norway, Sweden",Håndtering av udøde,0,no,https://trustnordisk.com/movie/handling-the-un...,"On a hot summer day in Oslo, the dead mysterio...",280.446,...,‘Handling the Undead’ Review: When the Dead Do...,Critic’s Pick,None,Dead Loved Ones Have Returned. Don’t Rejoice Yet.,None,None,None,By Alissa Wilkinson,"[{'firstname': 'Alissa', 'middlename': None, '...",None
2,Challengers,"Romance, Drama","English, Romanian","Italy, United States of America",Challengers,55000000,en,https://www.amazon.com/salp/challengers,Tennis player turned coach Tashi has taken her...,128.738,...,"‘Challengers’ Review: Game, Set, Love Matches",Critic’s pick,None,"Triple Fault: Game, Set, Love Matches",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
3,Unsung Hero,"Drama, Music, Family",English,United States of America,Unsung Hero,6000000,en,https://unsunghero.movie/,When David Smallbone's successful music compan...,87.539,...,‘Unsung Hero’ Review: Music Dedicated to the O...,None,None,Unsung Hero,None,None,None,By Nicolas Rapold,"[{'firstname': 'Nicolas', 'middlename': None, ...",None
4,Love Lies Bleeding,"Crime, Romance, Thriller",English,"United Kingdom, United States of America",Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,189.736,...,"‘Love Lies Bleeding’ Review: Kristen Stewart, ...",Critic’s Pick,None,"Bad Romance, Hard-Boiled Style",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [21]:
# Drop "byline.person" column
combined_df.drop(inplace=True, columns=['byline.person'])
combined_df.head(5)

,title,genres,spoken_languages,production_countries,original_title,budget,original_language,homepage,overview,popularity,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,Fly Me to the Moon,"Comedy, Romance",English,United States of America,Fly Me to the Moon,100000000,en,https://www.flymetothemoon.movie,Sparks fly in all directions as marketing mave...,89.170,...,525,‘Fly Me to the Moon’ Review: This NASA Rom-Com...,None,None,"It’s Love, NASA Style, But Earthbound in the End",None,None,None,By Glenn Kenny,None
1,Handling the Undead,"Horror, Drama, Mystery","Norwegian, Swedish","Greece, Norway, Sweden",Håndtering av udøde,0,no,https://trustnordisk.com/movie/handling-the-un...,"On a hot summer day in Oslo, the dead mysterio...",280.446,...,824,‘Handling the Undead’ Review: When the Dead Do...,Critic’s Pick,None,Dead Loved Ones Have Returned. Don’t Rejoice Yet.,None,None,None,By Alissa Wilkinson,None
2,Challengers,"Romance, Drama","English, Romanian","Italy, United States of America",Challengers,55000000,en,https://www.amazon.com/salp/challengers,Tennis player turned coach Tashi has taken her...,128.738,...,1029,"‘Challengers’ Review: Game, Set, Love Matches",Critic’s pick,None,"Triple Fault: Game, Set, Love Matches",None,None,None,By Manohla Dargis,None
3,Unsung Hero,"Drama, Music, Family",English,United States of America,Unsung Hero,6000000,en,https://unsunghero.movie/,When David Smallbone's successful music compan...,87.539,...,276,‘Unsung Hero’ Review: Music Dedicated to the O...,None,None,Unsung Hero,None,None,None,By Nicolas Rapold,None
4,Love Lies Bleeding,"Crime, Romance, Thriller",English,"United Kingdom, United States of America",Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,189.736,...,986,"‘Love Lies Bleeding’ Review: Kristen Stewart, ...",Critic’s Pick,None,"Bad Romance, Hard-Boiled Style",None,None,None,By Manohla Dargis,None


In [22]:
# Delete duplicate rows and reset index
# combined_df.drop_duplicates(inplace=True)
# , ignore_index=True)

combined_df.loc[combined_df.astype(str).drop_duplicates().index]
combined_df.reset_index(drop=True, inplace=True)
combined_df.head(5)

,title,genres,spoken_languages,production_countries,original_title,budget,original_language,homepage,overview,popularity,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,Fly Me to the Moon,"Comedy, Romance",English,United States of America,Fly Me to the Moon,100000000,en,https://www.flymetothemoon.movie,Sparks fly in all directions as marketing mave...,89.170,...,525,‘Fly Me to the Moon’ Review: This NASA Rom-Com...,None,None,"It’s Love, NASA Style, But Earthbound in the End",None,None,None,By Glenn Kenny,None
1,Handling the Undead,"Horror, Drama, Mystery","Norwegian, Swedish","Greece, Norway, Sweden",Håndtering av udøde,0,no,https://trustnordisk.com/movie/handling-the-un...,"On a hot summer day in Oslo, the dead mysterio...",280.446,...,824,‘Handling the Undead’ Review: When the Dead Do...,Critic’s Pick,None,Dead Loved Ones Have Returned. Don’t Rejoice Yet.,None,None,None,By Alissa Wilkinson,None
2,Challengers,"Romance, Drama","English, Romanian","Italy, United States of America",Challengers,55000000,en,https://www.amazon.com/salp/challengers,Tennis player turned coach Tashi has taken her...,128.738,...,1029,"‘Challengers’ Review: Game, Set, Love Matches",Critic’s pick,None,"Triple Fault: Game, Set, Love Matches",None,None,None,By Manohla Dargis,None
3,Unsung Hero,"Drama, Music, Family",English,United States of America,Unsung Hero,6000000,en,https://unsunghero.movie/,When David Smallbone's successful music compan...,87.539,...,276,‘Unsung Hero’ Review: Music Dedicated to the O...,None,None,Unsung Hero,None,None,None,By Nicolas Rapold,None
4,Love Lies Bleeding,"Crime, Romance, Thriller",English,"United Kingdom, United States of America",Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,189.736,...,986,"‘Love Lies Bleeding’ Review: Kristen Stewart, ...",Critic’s Pick,None,"Bad Romance, Hard-Boiled Style",None,None,None,By Manohla Dargis,None


In [23]:
# Export data to CSV without the index
combined_df.to_csv('./output/combined_data.csv', encoding='utf-8', index=False)